In [14]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [15]:
def extract_series(df):
    df_temp = df
    # Convert 'id' column to string type
    df_temp['id'] = df_temp['id'].astype(str)
    # Add a new column 'series' which is the first three digits of 'id'
    df_temp['series'] = df_temp['id'].str[:3].astype(int)

    # Read the series.csv file
    series = pd.read_csv('Series Name.csv')

    # Merge data with series on the 'series' column from data and 'series_id' column from series
    df_temp_1 = df_temp.merge(series, left_on='series', right_on='Series Name ID', how='left')

    cols = df_temp_1.columns.tolist()
    cols.insert(1, cols.pop(cols.index('Series Name')))
    df_temp_1 = df_temp_1[cols]
    df_temp_1.drop(columns=['Series Name ID', 'series'], inplace=True)

    return df_temp_1

In [16]:
def extract_contry(df):
    df_temp = df
    # Add a new column 'country' which is the fourth and fifth digits of 'id'
    df_temp['country'] = df_temp['id'].str[3:5].astype(int)

# Read the country.csv file
    country = pd.read_csv('Country Name.csv')

# Merge data_series with country on the 'country' column from data_series and 'country_id' column from country
    df_temp_1 = df_temp.merge(country, left_on='country', right_on='Country Name ID', how='left')

# Insert 'location' column into the correct position
    cols = df_temp_1.columns.tolist()
    cols.insert(2, cols.pop(cols.index('Country Name')))
    df_temp_1 = df_temp_1[cols]

# Drop unnecessary columns
    df_temp_1.drop(columns=['Country Name ID', 'country'], inplace=True)
    
    return df_temp_1

In [17]:
def extract_category(df):
    df_temp = df
    # Add a new column 'country' which is the fourth and fifth digits of 'id'
    df_temp['category'] = df_temp['id'].str[5:7].astype(int)

# Read the country.csv file
    category = pd.read_excel('category_id.xlsx')

# Rename the 'id' column in the country dataframe to 'country_id'
    category.rename(columns={'id': 'category_id'}, inplace=True)

# Merge data_series with country on the 'country' column from data_series and 'country_id' column from country
    df_temp_1 = df_temp.merge(category, left_on='category', right_on='category_id', how='left')

# Insert 'location' column into the correct position
    cols = df_temp_1.columns.tolist()
    cols.insert(3, cols.pop(cols.index('Category')))
    df_temp_1 = df_temp_1[cols]

# Drop unnecessary columns
    df_temp_1.drop(columns=['category_id', 'category'], inplace=True)

    return df_temp_1

In [18]:
qol = pd.read_csv('qol.csv')
qol.columns = ['year', 'qol']

In [19]:
data = pd.read_csv('cleaned_data.csv')
data['id'] = data['id'].astype(str)
data['id'] = data['id'].apply(lambda x: f'{int(x):013}' if pd.notnull(x) else x)
nf_data = data[data['id'].str[5:7] != '05']
nf_data = nf_data.set_index('id')

In [20]:
# Create a dataframe to store the id and p-values
pvalues_df = pd.DataFrame(columns=['id', 'pvalue'])

for col in range(len(nf_data)):

    # Ensure y and x have the same length
    x = nf_data.iloc[col].values.reshape(-1, 1)
    y = qol['qol']

    # Add a constant to the model (intercept)
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()

    pvalues_df.loc[col, 'id'] = nf_data.index[col]
    
    # Check if model.pvalues has an index 1
    if len(model.pvalues) > 1:
        pvalues_df.loc[col, 'pvalue'] = model.pvalues[1]  # Extract the p-value for the predictor variable
    else:
        pvalues_df.loc[col, 'pvalue'] = 1 # Assign NaN if p-value is not available



/var/folders/qz/xwx1r5sx2k9dgf6nxmgph3200000gn/T/ipykernel_36697/1575817326.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pvalues_df.loc[col, 'pvalue'] = model.pvalues[1]  # Extract the p-value for the predictor variable
/var/folders/qz/xwx1r5sx2k9dgf6nxmgph3200000gn/T/ipykernel_36697/1575817326.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pvalues_df.loc[col, 'pvalue'] = model.pvalues[1]  # Extract the p-value for the predictor variable
/var/folders/qz/xwx1r5sx2k9dgf6nxmgph3200000gn/T/ipykernel_36697/1575817326.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a futu

In [21]:
pvalues_threshold = 0.05 / len(nf_data)
nf_data_significant = pvalues_df[pvalues_df['pvalue'] < pvalues_threshold]

In [22]:
# Convert 'pvalue' column to numeric
pvalues_df['pvalue'] = pd.to_numeric(pvalues_df['pvalue'], errors='coerce')

# Get the top 10 rows with the smallest p-values
#nf_data_top10_pvalues = pvalues_df.nsmallest(100, 'pvalue')
nf_data_top10_pvalues = nf_data_significant

In [23]:
data = pd.read_csv('cleaned_data.csv')
data['id'] = data['id'].astype(str)
data['id'] = data['id'].apply(lambda x: f'{int(x):013}' if pd.notnull(x) else x)

nf_data_top10 = data[data['id'].isin(nf_data_top10_pvalues['id'])]

In [24]:
data_series = extract_series(nf_data_top10)
data_country = extract_contry(data_series)
data_category = extract_category(data_country)

/var/folders/qz/xwx1r5sx2k9dgf6nxmgph3200000gn/T/ipykernel_36697/571456676.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['id'] = df_temp['id'].astype(str)
/var/folders/qz/xwx1r5sx2k9dgf6nxmgph3200000gn/T/ipykernel_36697/571456676.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['series'] = df_temp['id'].str[:3].astype(int)


In [25]:
data_category.to_excel('top10.xlsx')

In [26]:
data_category.groupby('Series Name').count()['id'].sort_values(ascending=False).to_csv('top10_count.csv')